In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
import warnings
warnings.filterwarnings("ignore")

# 1. Data Preparation

In [3]:
## input training set ## 90960 x 16 df
df_ROI_train = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\modeling\\df_train_raMSIn_STDnorm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [4]:
## input ext val set ## 6075 x 16 df
df_ROI_ext = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\modeling\\df_ext_raMSIn_STDnorm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [5]:
## input FNA set ## 88701 x 16 df
df_ROI_FNA = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\modeling\\df_FNA_raMSIn_STDnorm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [6]:
## input DirectInfusion set ## 6057 x 16 df
df_ROI_Di = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\modeling\\df_Di_raMSIn_STDnorm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [7]:
## input DirectInfusion set ## 6711 x 16 df
df_ROI_C = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\modeling\\df_Clinics3_raMSIn_STDnorm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [8]:
df_ROI_Di

,pixel_id,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,514.2846,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.402342,0.913244,0.833746,-0.387446,0.488453,1.697873,0.601978,0.518836,2.971000,0.532073,0,0,0,1,TN
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.314493,1.539995,0.003972,0.129889,0.246011,1.609584,0.390074,0.742545,2.805791,0.443193,0,0,0,1,TN
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.841655,1.554217,0.670368,0.339209,0.950243,2.182584,1.755656,0.796962,3.286110,0.989125,0,0,0,1,TN
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.621964,2.090087,0.272553,0.593092,1.079633,1.903016,0.849887,0.596528,3.016573,0.801811,0,0,0,1,TN
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.303707,1.234845,0.156936,0.247718,0.548905,1.677419,0.944668,0.913738,3.116293,0.783867,0,0,0,1,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3023,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,0.046377,0.004282,1.384092,0.640055,1.053762,-0.172416,0.489131,0.529506,-0.437109,1.369131,1,1,1,0,TP
3024,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-1.799346,-1.043795,-1.051923,-1.271685,0.599189,-1.114427,1.334575,-1.475923,0.064193,0.404445,1,1,1,0,TP
3025,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.519249,-1.667011,1.280725,-0.144515,0.109578,-0.360029,0.180897,0.768313,-0.437109,1.725927,1,1,1,0,TP
3026,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.419602,-0.812576,1.794612,-0.889368,0.995031,-0.357697,0.403818,-0.280928,-0.437109,1.301959,1,1,1,0,TP


In [9]:
df_ROI_C

,pixel_id,311.1684,269.2486,215.0324,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,448.3072,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.180725,-0.876131,-0.053859,-0.672764,-1.247431,-0.561901,-1.461826,-0.434488,-0.118059,-0.600421,0,1,1,0,FP
1,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.357559,-0.557249,0.071199,-0.672764,-0.885699,0.896613,-1.461826,-0.434488,0.141921,-0.600421,0,1,1,0,FP
2,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.360496,-0.732288,0.443584,-0.672764,-1.247431,0.873763,-1.461826,-0.434488,0.228312,-0.600421,0,1,1,0,FP
3,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.414136,-0.641708,0.276798,-0.672764,-1.247431,0.807055,-1.461826,-0.434488,0.188690,-0.600421,0,1,1,0,FP
4,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.291888,-0.550431,0.173477,-0.672764,-1.247431,0.742037,-1.461826,1.201794,0.099504,-0.600421,0,1,1,0,FP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6706,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_27_24,-0.533442,-1.203196,-0.483954,-0.672764,-1.247431,-0.561901,-1.461826,-0.434488,-1.551766,-0.600421,1,1,1,0,TP
6707,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_33_25,-0.510905,-1.337545,-0.483954,-0.672764,-1.247431,-0.561901,-0.463838,-0.434488,-1.930156,-0.600421,1,1,1,0,TP
6708,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_34_25,-0.272450,-1.091268,-0.483954,-0.672764,-1.247431,-0.561901,-0.498383,-0.434488,-1.733144,-0.600421,1,1,1,0,TP
6709,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_35_25,-0.445932,-1.194061,-0.483954,-0.672764,-1.247431,-0.561901,-0.549042,-0.434488,-1.408482,-0.600421,1,1,1,0,TP


In [10]:
df_ROI_train["Sample"] = 0
df_ROI_train["Row"] = 0
df_ROI_train["Scan"] = 0
df_ROI_train_Sample = []
df_ROI_train_Row = []
df_ROI_train_Scan = []
for i in range(len(df_ROI_train["pixel_id"])):
    df_ROI_train_Sample.append("_".join(str(df_ROI_train["pixel_id"][i]).split("_")[:-2]))
    df_ROI_train_Row.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-1]))
    df_ROI_train_Scan.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-2]))
df_ROI_train["Sample"] = df_ROI_train_Sample
df_ROI_train["Row"] = df_ROI_train_Row
df_ROI_train["Scan"] = df_ROI_train_Scan
pop_column = df_ROI_train.pop('Scan')
df_ROI_train.insert(1, 'Scan', pop_column)
pop_column = df_ROI_train.pop('Row')
df_ROI_train.insert(1, 'Row', pop_column)
pop_column = df_ROI_train.pop('Sample')
df_ROI_train.insert(1, 'Sample', pop_column)

df_ROI_ext["Sample"] = 0
df_ROI_ext["Row"] = 0
df_ROI_ext["Scan"] = 0
df_ROI_ext_Sample = []
df_ROI_ext_Row = []
df_ROI_ext_Scan = []
for i in range(len(df_ROI_ext["pixel_id"])):
    df_ROI_ext_Sample.append("_".join(str(df_ROI_ext["pixel_id"][i]).split("_")[:-2]))
    df_ROI_ext_Row.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-1]))
    df_ROI_ext_Scan.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-2]))
df_ROI_ext["Sample"] = df_ROI_ext_Sample
df_ROI_ext["Row"] = df_ROI_ext_Row
df_ROI_ext["Scan"] = df_ROI_ext_Scan
pop_column = df_ROI_ext.pop('Scan')
df_ROI_ext.insert(1, 'Scan', pop_column)
pop_column = df_ROI_ext.pop('Row')
df_ROI_ext.insert(1, 'Row', pop_column)
pop_column = df_ROI_ext.pop('Sample')
df_ROI_ext.insert(1, 'Sample', pop_column)

df_ROI_FNA["Sample"] = 0
df_ROI_FNA["Row"] = 0
df_ROI_FNA["Scan"] = 0
df_ROI_FNA_Sample = []
df_ROI_FNA_Row = []
df_ROI_FNA_Scan = []
for i in range(len(df_ROI_FNA["pixel_id"])):
    df_ROI_FNA_Sample.append("_".join(str(df_ROI_FNA["pixel_id"][i]).split("_")[:-2]))
    df_ROI_FNA_Row.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-1]))
    df_ROI_FNA_Scan.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-2]))
df_ROI_FNA["Sample"] = df_ROI_FNA_Sample
df_ROI_FNA["Row"] = df_ROI_FNA_Row
df_ROI_FNA["Scan"] = df_ROI_FNA_Scan
pop_column = df_ROI_FNA.pop('Scan')
df_ROI_FNA.insert(1, 'Scan', pop_column)
pop_column = df_ROI_FNA.pop('Row')
df_ROI_FNA.insert(1, 'Row', pop_column)
pop_column = df_ROI_FNA.pop('Sample')
df_ROI_FNA.insert(1, 'Sample', pop_column)

df_ROI_Di["Sample"] = 0
df_ROI_Di["Row"] = 0
df_ROI_Di["Scan"] = 0
df_ROI_Di_Sample = []
df_ROI_Di_Row = []
df_ROI_Di_Scan = []
for i in range(len(df_ROI_Di["pixel_id"])):
    df_ROI_Di_Sample.append("_".join(str(df_ROI_Di["pixel_id"][i]).split("_")[:-2]))
    df_ROI_Di_Row.append(int(str(df_ROI_Di["pixel_id"][i]).split("_")[-1]))
    df_ROI_Di_Scan.append(int(str(df_ROI_Di["pixel_id"][i]).split("_")[-2]))
df_ROI_Di["Sample"] = df_ROI_Di_Sample
df_ROI_Di["Row"] = df_ROI_Di_Row
df_ROI_Di["Scan"] = df_ROI_Di_Scan
pop_column = df_ROI_Di.pop('Scan')
df_ROI_Di.insert(1, 'Scan', pop_column)
pop_column = df_ROI_Di.pop('Row')
df_ROI_Di.insert(1, 'Row', pop_column)
pop_column = df_ROI_Di.pop('Sample')
df_ROI_Di.insert(1, 'Sample', pop_column)

In [11]:
df_ROI_C["Sample"] = 0
df_ROI_C["Row"] = 0
df_ROI_C["Scan"] = 0
df_ROI_C_Sample = []
df_ROI_C_Row = []
df_ROI_C_Scan = []
for i in range(len(df_ROI_C["pixel_id"])):
    df_ROI_C_Sample.append("_".join(str(df_ROI_C["pixel_id"][i]).split("_")[:-2]))
    df_ROI_C_Row.append(int(str(df_ROI_C["pixel_id"][i]).split("_")[-1]))
    df_ROI_C_Scan.append(int(str(df_ROI_C["pixel_id"][i]).split("_")[-2]))
df_ROI_C["Sample"] = df_ROI_C_Sample
df_ROI_C["Row"] = df_ROI_C_Row
df_ROI_C["Scan"] = df_ROI_C_Scan
pop_column = df_ROI_C.pop('Scan')
df_ROI_C.insert(1, 'Scan', pop_column)
pop_column = df_ROI_C.pop('Row')
df_ROI_C.insert(1, 'Row', pop_column)
pop_column = df_ROI_C.pop('Sample')
df_ROI_C.insert(1, 'Sample', pop_column)

In [12]:
df_ROI_train = df_ROI_train.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_ext = df_ROI_ext.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_FNA = df_ROI_FNA.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_Di = df_ROI_Di.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_C = df_ROI_C.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [13]:
df_ROI_Di

,pixel_id,Sample,Row,Scan,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,514.2846,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,2,0.402342,0.913244,0.833746,-0.387446,0.488453,1.697873,0.601978,0.518836,2.971000,0.532073,0,0,0,1,TN
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,406,0.593824,2.184017,0.506205,0.906626,0.098951,1.829089,1.031092,1.224294,2.831356,0.765853,0,0,0,1,TN
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,400,0.050242,2.114770,0.491892,0.058591,0.162173,1.494161,0.722599,0.985018,2.796233,0.226743,0,0,0,1,TN
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,398,-0.133182,2.044132,-0.250392,0.497755,0.441963,1.516943,1.486171,0.407406,2.427185,0.287358,0,0,0,1,TN
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,396,-0.020739,2.563432,-0.376681,0.464466,0.063178,1.363749,0.281264,0.833601,2.559720,0.323531,0,0,0,1,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3023,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,245,-1.058011,-0.908671,0.717646,-0.556439,0.639557,-0.486346,0.284608,0.949616,-0.437109,1.033031,1,1,1,0,TP
3024,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,244,-1.055232,-1.000239,0.925506,-0.229124,-0.826830,-0.853034,-0.137459,0.912807,-0.437109,1.225038,1,1,1,0,TP
3025,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,242,-0.782721,-0.682543,2.062912,-0.412998,0.244065,-0.506230,0.202932,1.081918,-0.437109,1.697405,1,1,1,0,TP
3026,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,250,-0.892271,-0.872716,1.189912,-0.433834,-0.018556,-0.470024,-0.326227,0.638063,-0.437109,1.367409,1,1,1,0,TP


In [14]:
df_ROI_C

,pixel_id,Sample,Row,Scan,311.1684,269.2486,215.0324,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,448.3072,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1,4,19,0.180725,-0.876131,-0.053859,-0.672764,-1.247431,-0.561901,-1.461826,-0.434488,-0.118059,-0.600421,0,1,1,0,FP
1,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1,17,20,3.430032,1.590122,0.381000,1.436097,1.056724,2.052369,2.149254,-0.434488,1.823848,2.591255,0,1,1,0,FP
2,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1,17,19,3.518585,1.752746,-0.483954,1.451756,0.922312,1.953356,1.961649,1.461844,1.800219,2.478244,0,0,0,1,TN
3,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1,17,18,3.802304,1.780981,-0.483954,1.354415,1.027931,2.183731,1.797010,2.114303,1.614046,2.830831,0,0,0,1,TN
4,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1,17,17,5.767832,2.469604,1.030213,1.450765,1.141729,3.919320,1.884768,2.143488,1.575992,2.593630,0,0,0,1,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6706,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_18_10,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7,10,18,0.321729,-0.495496,-0.483954,0.253447,-0.229910,0.829892,1.695432,-0.434488,-1.034082,-0.600421,1,0,0,1,FN
6707,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_17_10,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7,10,17,-0.129185,-0.587068,-0.483954,-0.672764,-0.220462,-0.561901,1.371458,-0.434488,-0.910235,-0.600421,1,1,1,0,TP
6708,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_16_10,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7,10,16,0.108741,-0.598898,-0.483954,-0.672764,-0.327677,0.834525,1.348215,-0.434488,-1.367925,-0.600421,1,1,1,0,TP
6709,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_14_9,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7,9,14,0.393596,-0.379988,-0.483954,-0.672764,-0.034011,0.744596,1.207819,-0.434488,-1.177437,-0.600421,1,1,1,0,TP


In [15]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [16]:
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_ROI = df_ROI_C.loc[df_ROI_C["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [74]:
sample_ROI_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI]

                   #HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1_ROI, 
                   #HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2_ROI, 
                   #HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3_ROI, 
                   #HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4_ROI, 
                   #HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5_ROI, 
                   #HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6_ROI, 
                   #HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7_ROI, 
                   #HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1_ROI, 
                   #HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2_ROI, 
                   #HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3_ROI, 
                   #HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4_ROI, 
                   #HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5_ROI, 
                   #HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6_ROI, 
                   #HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_ROI]

In [75]:
name_list = ["HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI"]

             #"HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1_ROI", 
             #"HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2_ROI", 
             #"HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3_ROI", 
             #"HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4_ROI", 
             #"HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5_ROI", 
             #"HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6_ROI", 
             #"HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7_ROI", 
             #"HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1_ROI", 
             #"HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2_ROI", 
             #"HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3_ROI", 
             #"HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4_ROI", 
             #"HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5_ROI", 
             #"HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6_ROI", 
             #"HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_ROI"]

In [76]:
sample_ROI_list_train = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI]

sample_ROI_list_ext = [HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_ingested = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_FNA = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI]

sample_ROI_list_DirectIn = [HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI]

sample_ROI_list_ClinicsMSI = [HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1_ROI, 
                              HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2_ROI, 
                              HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3_ROI, 
                              HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4_ROI, 
                              HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5_ROI, 
                              HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6_ROI, 
                              HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7_ROI, 
                              HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1_ROI, 
                              HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2_ROI, 
                              HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3_ROI, 
                              HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4_ROI, 
                              HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5_ROI, 
                              HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6_ROI, 
                              HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_ROI]

In [77]:
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI

,pixel_id,Sample,Row,Scan,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,514.2846,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,4,3.558456,-1.182921,1.125065,0.222380,-0.076987,2.807953,-2.359817,-1.125419,0.536431,2.211504,0,1,1,0,FP
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,5,-0.177356,0.501077,1.087192,0.816910,-0.198743,2.620396,0.033703,0.185933,1.463984,1.637294,0,0,0,1,TN
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,6,-0.177356,0.736289,1.113186,0.425437,-0.395288,2.445937,-2.359817,0.365792,1.778802,2.226856,0,0,0,1,TN
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,194,-0.177356,4.811911,2.007718,5.326192,0.069331,8.667370,1.159799,2.370673,6.400470,5.532763,0,0,0,1,TN
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,195,-0.177356,4.376351,2.353939,5.929659,-0.757070,8.809192,1.458943,1.859186,6.727145,7.013887,0,0,0,1,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,144,-0.177356,0.827793,0.548668,0.679038,0.270861,1.165109,-0.106640,1.327304,-0.596690,0.790571,0,0,0,1,TN
14821,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,145,-0.177356,0.803773,0.678295,1.280821,-0.392594,-0.485628,-0.438487,1.021118,0.059893,1.474085,0,0,0,1,TN
14822,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,146,-0.177356,0.708635,0.790757,1.389889,-1.051298,-0.485628,0.280468,1.056665,-0.246461,0.832943,0,0,0,1,TN
14823,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,118,177,-0.177356,3.501409,-1.017803,2.625022,-1.324608,3.045150,-0.734281,0.402018,3.911239,-0.523816,0,0,0,1,TN


In [78]:
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI

,pixel_id,Sample,Row,Scan,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,514.2846,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_5_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,5,23.652547,1.520263,-1.017803,0.475729,0.054166,11.538737,-0.579021,-1.125419,3.696491,-0.523816,1,1,1,0,TP
1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_6_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,6,19.377898,2.021440,-1.017803,-1.412056,0.946499,9.417823,-0.136716,-1.125419,4.306018,-0.523816,1,1,1,0,TP
2,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_7_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,7,18.033572,2.126559,-1.017803,0.449718,1.023653,7.896305,0.456410,-1.125419,4.891705,-0.523816,1,1,1,0,TP
3,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_8_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,8,18.799716,2.645661,-1.017803,0.498729,0.005613,8.223050,0.237898,-1.125419,4.758369,-0.523816,1,1,1,0,TP
4,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_9_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,9,20.426014,2.273658,-1.017803,0.907228,-0.059125,9.330800,-0.029527,-1.125419,5.177388,-0.523816,1,1,1,0,TP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14249,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,189,6.335743,2.714414,-1.017803,1.989273,-0.184236,-0.485628,-2.359817,-1.125419,5.244023,-0.523816,1,1,1,0,TP
14250,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,190,6.492199,2.498405,-1.017803,1.632040,0.432772,-0.485628,-2.359817,-1.125419,5.463969,-0.523816,1,1,1,0,TP
14251,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,191,6.357034,2.605986,-1.017803,1.804683,-0.038264,-0.485628,-2.359817,-1.125419,6.001737,-0.523816,1,1,1,0,TP
14252,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,126,185,6.292874,2.818997,-1.017803,1.850685,0.780029,-0.485628,-2.359817,-1.125419,5.933882,-0.523816,1,1,1,0,TP


# 2. Prediction (made)

## 2.1. FNA

In [79]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(14.833052276559865, '%')

In [80]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(69.81953951462351, '%')

In [81]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["predicted0n1"]).count(1)) ) *100, "%"

(27.793076317859956, '%')

In [82]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(91.65146625508629, '%')

In [83]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["predicted0n1"]).count(1)) ) *100, "%"

(88.74852502255848, '%')

In [84]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["predicted0n1"]).count(1)) ) *100, "%"

(95.88387096774194, '%')

## 2.2. Direct Infusion

In [90]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI["predicted0n1"]).count(1)) ) *100, "%"

(77.88778877887789, '%')

In [91]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI["predicted0n1"]).count(1)) ) *100, "%"

(97.6897689768977, '%')

In [92]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI["predicted0n1"]).count(1)) ) *100, "%"

(88.44884488448845, '%')

In [93]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI["predicted0n1"]).count(1)) ) *100, "%"

(88.44884488448845, '%')

In [94]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI["predicted0n1"]).count(1)) ) *100, "%"

(91.0891089108911, '%')

## 2.3. Clinics MSI

In [102]:
( list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1_ROI["predicted0n1"]).count(1)) ) *100, "%"

(84.93449781659389, '%')

In [103]:
( list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2_ROI["predicted0n1"]).count(1)) ) *100, "%"

(65.23076923076923, '%')

In [104]:
( list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(62.236286919831215, '%')

In [105]:
( list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4_ROI["predicted0n1"]).count(1)) ) *100, "%"

(80.0, '%')

In [106]:
( list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5_ROI["predicted0n1"]).count(1)) ) *100, "%"

(80.91503267973856, '%')

In [107]:
( list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6_ROI["predicted0n1"]).count(1)) ) *100, "%"

(79.35483870967742, '%')

In [108]:
( list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_ROI["predicted0n1"]).count(1)) ) *100, "%"

(94.44444444444444, '%')

# 3. EDA

In [109]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [110]:
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm
import numpy as np
from matplotlib.colors import ListedColormap
cmap = cm.binary

def rgb_white2alpha(rgb, ensure_increasing=False):

    # The most transparent alpha we can use is given by the min of RGB
    # Convert it from saturation to opacity
    alpha = 1. - np.min(rgb, axis=1)
    if ensure_increasing:
        # Let's also ensure the alpha value is monotonically increasing
        a_max = alpha[0]
        for i, a in enumerate(alpha):
            alpha[i] = a_max = np.maximum(a, a_max)
    alpha = np.expand_dims(alpha, -1)
    # Rescale colors to discount the white that will show through from transparency
    rgb = (rgb + alpha - 1) / alpha
    # Concatenate our alpha channel
    return np.concatenate((rgb, alpha), axis=1)
    

def cmap_white2alpha(name, ensure_increasing=False, register=True):

    # Fetch the cmap callable
    cmap = plt.get_cmap(name)
    # Get the colors out from the colormap LUT
    rgb = cmap(np.arange(cmap.N))[:, :3]  # N-by-3
    # Convert white to alpha
    rgba = rgb_white2alpha(rgb, ensure_increasing=ensure_increasing)
    # Create a new Colormap object
    cmap_alpha = matplotlib.colors.ListedColormap(rgba, name=name + "_alpha")
    if register:
        matplotlib.cm.register_cmap(name=name + "_alpha", cmap=cmap_alpha)
    return cmap_alpha

In [146]:
# Get our new version of the colormap with white2alpha
cmap_w2a = cmap_white2alpha("Blues")

In [147]:
def genAROItif(sample_ROI, sample_, name, mz, vmax_=1):

    df_heatmap = sample_ROI[["Row", "Scan", mz]]
    df_heatmap2 = pd.DataFrame(np.array([[0, 0, 0]]), columns=['Row', 'Scan', mz]).reset_index().drop(columns = ["index"])
    df_HM = df_heatmap2.loc[df_heatmap["Row"] == 0].loc[df_heatmap2["Scan"] == 0]
    Heat_Row = sample_["Row"]
    Heat_Scan = sample_["Scan"]


    #hi = 0
    for y in range(len(set(Heat_Row))):
        for x in range(1, len(set(Heat_Scan))+1):

            df_in = df_heatmap.loc[df_heatmap["Row"] == (y+1)].loc[df_heatmap["Scan"] == x]
            if len(df_in) == 1:
                df_HM = pd.concat([df_HM, df_in], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


            elif len(df_in) == 0:
                df_heatmap2["Row"] = y+1
                df_heatmap2["Scan"] = x
                df_HM = pd.concat([df_HM, df_heatmap2], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


    #print(name, mz, hi, len(df_HM))
    if "index" in df_HM.columns:
        df_HM = df_HM.drop(columns = ["index", "Row", "Scan"])
    else:
        df_HM = df_HM.drop(columns = ["Row", "Scan"])

    df_HM = df_HM.T
    New_y = []
    for y in range(1, len(set(Heat_Row))+1):
        New_y.append(str(y))
    New_x = []
    for x in range(1, len(set(Heat_Scan))+1):
        New_x.append(str(x))

    arr = df_HM.values.copy()
    arr.resize(len(set(Heat_Row)), len(set(Heat_Scan)))

    df_HM = pd.DataFrame(arr)
    df_HM.index = New_y
    df_HM.columns = New_x

    for i in list(df_HM.columns):
        df_HM[i] = df_HM[i].astype(float)

    plt.subplots(dpi = 300)
    plt.imshow(df_HM, cmap=cmap_w2a, vmin=0, vmax=1, zorder=1) #cmap_w2a
    plt.xticks([])
    plt.yticks([])
    plt.box(False)
    plt.autoscale(enable=True, tight=True)

    plt.rcParams['figure.figsize'] = [2, 2]

    plt.savefig(f"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/raMSIn/eda/{name}_{mz}_0.tif", bbox_inches = 'tight', pad_inches = 0)

In [135]:
df_raMSI_ML = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/raMSI_inFNA_noLT.csv').set_index('pixel_id')
df_raMSI_ML.shape

(371480, 2554)

df_raMSI_ML = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_NonInClinicsMSI/raMSIn_ClinicsMSI_noLT.csv').set_index('pixel_id')
df_raMSI_ML.shape

In [136]:
raMSI_ML = df_raMSI_ML.reset_index()

In [137]:
raMSI_ML["Sample"] = 0
raMSI_ML["Row"] = 0
raMSI_ML["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(raMSI_ML["pixel_id"])):
    Sample.append("_".join(str(raMSI_ML["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-2]))
raMSI_ML["Sample"] = Sample
raMSI_ML["Row"] = Row
raMSI_ML["Scan"] = Scan

In [138]:
pop_column = raMSI_ML.pop('Scan')
raMSI_ML.insert(1, 'Scan', pop_column)
pop_column = raMSI_ML.pop('Row')
raMSI_ML.insert(1, 'Row', pop_column)
pop_column = raMSI_ML.pop('Sample')
raMSI_ML.insert(1, 'Sample', pop_column)

In [139]:
raMSI_ML = raMSI_ML.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [140]:
raMSI_ML

,pixel_id,Sample,Row,Scan,100.0041,100.926,100.9337,100.9449,100.9462,101.0232,...,1451.9885,1452.9964,1453.9998,1473.9761,1474.9842,1475.9932,1476.997,1478.006,1495.9578,type
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,100,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,0
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,107,90,0.0,0.000000,853.711548,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,0
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,108,90,0.0,0.000000,703.245972,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,0
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,109,90,0.0,0.000000,766.532532,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,528.50592,0.0,0.000000,0
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,11,90,0.0,919.982178,3530.654541,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371475,HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4...,HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4,24,22,0.0,0.000000,0.000000,0.0,0.0,1769.011963,...,867.352844,463.809998,83862.335938,0.000000,0.0,0.0,0.00000,0.0,0.000000,1
371476,HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4...,HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4,23,22,0.0,0.000000,0.000000,0.0,0.0,1665.778687,...,696.084412,0.000000,85221.218750,512.856140,0.0,0.0,0.00000,0.0,0.000000,1
371477,HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4...,HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4,22,22,0.0,0.000000,0.000000,0.0,0.0,1623.990479,...,797.990173,522.887573,100720.523438,1567.466675,0.0,0.0,0.00000,0.0,334.524353,1
371478,HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4...,HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4,29,22,0.0,0.000000,0.000000,0.0,0.0,529.313049,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,1


In [141]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [142]:
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2

,pixel_id,Sample,Row,Scan,100.0041,100.926,100.9337,100.9449,100.9462,101.0232,...,1451.9885,1452.9964,1453.9998,1473.9761,1474.9842,1475.9932,1476.997,1478.006,1495.9578,type
0,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_1_1,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2,1,1,0.0,3416.076172,5855.711914,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_2_1,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2,1,2,0.0,3800.449951,7218.336914,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_3_1,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2,1,3,0.0,4193.261719,7841.295898,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_4_1,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2,1,4,0.0,3170.118408,6399.361816,0.0,448.873688,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_5_1,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2,1,5,0.0,3411.152344,6106.943359,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30995,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_2...,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2,125,244,0.0,3250.485596,9221.105469,0.0,936.335632,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
30996,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_2...,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2,125,245,0.0,3060.825928,6637.722168,0.0,663.132812,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
30997,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_2...,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2,125,246,0.0,2076.545898,6887.074219,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
30998,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_2...,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2,125,247,0.0,2869.934326,7913.682617,0.0,858.891052,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [143]:
sample_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6, 
               HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2]

sample_list = [HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1, 
               HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S2, 
               HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S3, 
               HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S4, 
               HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S5, 
               HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S6, 
               HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S7, 
               HKULiver_Post29wk_HCCLiver_Clinics_Liver_S1, 
               HKULiver_Post29wk_HCCLiver_Clinics_Liver_S2, 
               HKULiver_Post29wk_HCCLiver_Clinics_Liver_S3, 
               HKULiver_Post29wk_HCCLiver_Clinics_Liver_S4, 
               HKULiver_Post29wk_HCCLiver_Clinics_Liver_S5, 
               HKULiver_Post29wk_HCCLiver_Clinics_Liver_S6, 
               HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7]

In [148]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

TOP ="F"
    

counter = 0

for im in range(len(sample_ROI_list[:-10])):
    start = datetime.now()

    name = name_list[im]
    sample_ = sample_list[im]
    preview = sample_ROI_list[im][[TOP]]
    for i in preview.columns:
        preview[i] = preview[i].astype(float)

    vmax_list = list(preview.describe().iloc[-1])
    vmax_ = 1
    #print(TOP, name, vmax_)

    genAROItif(sample_ROI=sample_ROI_list[im], sample_=sample_, name=name, mz=TOP, vmax_=vmax_)
    #print(df.shape)
    counter += 1
    end = datetime.now()
    Duration = end - start
    print(Duration)


0:00:59.525317
0:01:04.427860
0:01:15.383004
0:00:38.464900
0:00:48.245966
0:01:05.976698
0:00:03.549524
0:00:02.438534
0:00:03.877831
0:00:03.786250
0:02:28.455000
0:02:25.962856
0:02:41.495338
0:01:07.474126
0:00:53.870196
0:00:45.592943
0:00:45.044677
0:00:43.689108
0:00:44.471134
0:00:02.325704
0:00:05.970247
0:00:03.337616
0:00:03.125200
0:02:16.909359
0:02:18.955398
0:02:33.893986
